In [2]:
import math
import numpy as np
import matplotlib as plt
import turtle

In [ ]:
def init_positions(n):
    pass
def calculate_interagent_distance(positions):
    # Given : n x 2 matrix
    # return : n x n matrix
    
    pass

def calculate_interagent_theta(positions):
    # Given : n x 2 matrix
    # return : n x n matrix
    
    pass

def calculate_force(distance, Fmax, R):
    pass
def update_velocity(fx, fy, dt):
    return vx, vy
def update_position(vx, vy, dt):
    return x, y
def distance_to_move(dx, dy):
    pass
def bearing_angle(dx, dy):
    pass

    

In [ ]:
# initialise positions for all agents and agent velocities to zero and simulate
# calculate the distance, bearing of all robot for each robot - r,theta
# for each timestep:
    # For each robot do the following at each timestep:
        # for each neighbouring agent do the following:
            # calculate forces due to each robot
            # break into x,y component and calculate cummulative
        # calculate the updated velocity 
        # calculate new position of robot
        # calculate, turn and distance to move for each agent
    # simulate movement and set velocity back to zero(friction)
